# 1-3 GAN

<img src="./img/gan.png" alt="gan" width="500" align="left"/>

### 모듈 임포트
- os (디렉토리 생성)
- tensorflow (학습)
- numpy (랜덤데이터 플로팅)
- matplotlib.pyplot (플로팅)
- matplotlib.gridspec (플로팅)

In [ ]:
import os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import gridspec as gridspec

### Logging Directory 설정

In [ ]:
CKPT_DIR = '../generated_output/GAN'

### 학습 변수 설정
- learning rate : gradient descent시 적용할 학습률
- training steps : training 종료조건
- batch size : 1 step에 feed forward할 배치 크기

- 1 step에 batch size만큼의 데이터를 feed forward하기 때문에, step * batch size = feed forward된 데이터 샘플수
- 1 epoch은 총 sample수를 사용하여 학습한 것을 의미하므로, feed forward된 sample수 / 데이터 sample수 = epoch수

In [ ]:
tf.random.set_random_seed(10)

LEARNING_RATE = 1e-4
TRAINING_STEPS = 60000
BATCH_SIZE = 100

### 네트워크 변수 설정
- image dimension : 이미지 차원, Fashion-MNIST가 28x28이므로 총 784차원의 벡터
- noise dimension : 랜덤 샘플링할 차원
- generator hidden dimension : generator 파트의 은닉층 차원
- discriminator hidden dimension : discriminator 파트의 은닉층 차원

In [ ]:
IMAGE_DIM = 784
NOISE_DIM = 100
GEN_HIDDEN_DIM = 256
DISC_HIDDEN_DIM = 256
graph = tf.Graph()

### Discriminator 함수 정의

[batch_size, 784]

$\rightarrow$ Dense(784, 256) $\rightarrow$ relu $\rightarrow$ [batch_size, 256]

$\rightarrow$ Dense(256, 1) $\rightarrow$ sigmoid $\rightarrow$ [batch_size, 1]

#### Initialization 구현

shape = [784, 256]일 때, 이에 대한 weight initialization 구현법  
(실용 코드에서는 tensorflow. initializers에 함수로 구현되어, 호출만 하면 됨)

1. Lecun initialization

> tf.random_normal(shape=[shape], stddev=tf.sqrt(1. / (shape[0]))

2. Glorot initialization

> tf.random_normal(shape=[shape], stddev=tf.sqrt(2. / (shape[0] + shape[1])))

3. He initialization

> tf.random_normal(shape=[shape], stddev=tf.sqrt(2. / shape[0]))

In [ ]:
def disc_model(features):
    
    with tf.variable_scope('discriminator', reuse=tf.AUTO_REUSE):
    # variable scope 이용하여 선별적 weight update 진행

        net = features
        
        net = tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                         input_shape=[28, 28, 1], name='d_c_1')(net)
        net = tf.keras.layers.LeakyReLU()(net)
        net = tf.keras.layers.Dropout(0.3)(net)
        net = tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same', name='d_c_2')(net)
        net = tf.keras.layers.LeakyReLU()(net)
        net = tf.keras.layers.Dropout(0.3)(net)
        net = tf.keras.layers.Flatten()(net)
        net = tf.keras.layers.Dense(1, name='d_d_3')(net)
    
        return net
        # output layer을 거친 tensor을 리턴

### Generator 함수 정의

[batch_size, 100]

$\rightarrow$ Dense(100, 256) $\rightarrow$ relu $\rightarrow$ [batch_size, 256]

$\rightarrow$ Dense(256, 784) $\rightarrow$ sigmoid $\rightarrow$ [batch_size, 784]

In [ ]:
def gen_model(features):
    
    with tf.variable_scope('generator', reuse=tf.AUTO_REUSE):
    # variable scope 이용하여 선별적 weight update 진행
        
        net = features
        
        net = tf.keras.layers.Dense(7*7*256, use_bias=False, input_shape=(100,), name='g_d_1')(net)
        net = tf.keras.layers.BatchNormalization(name='g_b_1')(net)
        net = tf.keras.layers.LeakyReLU()(net)
        net = tf.keras.layers.Reshape((7, 7, 256))(net)
 
        net = tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False, name='g_c_2')(net)
        net = tf.keras.layers.BatchNormalization(name='g_b_2')(net)
        net = tf.keras.layers.LeakyReLU()(net)

        net = tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False, name='g_c_3')(net)
        net = tf.keras.layers.BatchNormalization(name='g_b_3')(net)
        net = tf.keras.layers.LeakyReLU()(net)

        net = tf.keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh', name='g_c_o')(net)
        
        return net
        # output layer을 거친 tensor을 리턴

### Input Function 정의

feature

$\rightarrow$ Dataset 생성 $\rightarrow$ 데이터 사이즈로 랜덤 셔플링 $\rightarrow$ 데이터 모두 소모시 처음부터 반복

$\rightarrow$ batch 생성 $\rightarrow$ batch 한 개씩 소모하는 iterator $\rightarrow$ iterator 호출

In [ ]:
def train_input_fn(features, batch_size=BATCH_SIZE):
    
    with graph.as_default():
        
        dataset = tf.data.Dataset.from_tensor_slices(features)
        # 인풋값으로부터 Dataset객체 생성
        batch_dataset = dataset.shuffle(features.shape[0]).repeat().batch(batch_size)
        # Dataset 셔플링, 반복, 배치화
        batch = batch_dataset.make_one_shot_iterator().get_next()
        # 단일 배치 반복자 리턴
        return batch

<img src="./img/gan_loss.png" alt="ganloss" width="800" align="left"/>

In [ ]:
def train(features):
    
    if not os.path.exists(os.path.dirname(CKPT_DIR)):
        os.makedirs(os.path.dirname(CKPT_DIR))
    # logging directory 생성
        
    with graph.as_default():
        
        features = train_input_fn(features)
        # feature 받아서 batch로 넘겨주는 그래프 작성

        real_image = features
        # real image는 입력받는 이미지 tensor
        
        fake_noise = tf.random.normal(
            shape=[BATCH_SIZE, NOISE_DIM], 
            dtype=tf.float32)
        # fake noise는 -1~1의 uniform distribution으로 sampling한 tensor
        
        fake_image = gen_model(fake_noise)
        # fake image는 fake nose를 generator에 적용하여 얻은 이미지
        
        disc_real = disc_model(real_image)
        # discriminator에 real image가 들어온 경우의 tensor
        
        disc_fake = disc_model(fake_image)
        # discriminator에 fake image가 들어온 경우의 tensor
        
        disc_real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(disc_real), disc_real)
        disc_fake_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(disc_fake), disc_fake)
        disc_loss = disc_real_loss + disc_fake_loss
        # discriminator loss (상기 그림 중 좌측)
        # discriminator 끝단에 sigmoid가 붙어있기에 0~1로 mapping됨
        # real image는 1로 판정하고자 함 (-log(1) = 0, -log(0) = inf)
        # fake image는 0으로 판정하고자 함 (-log(1-1) = inf, -log(1-0) = 0)
        
        gen_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(disc_fake), disc_fake)
        # generator loss (상기 그림 중 우측)
        # gen_loss = tf.reduce_mean(tf.log(1. - disc_fake)) 의 대체 로스 (gradient vanishing 방지)
        # real값에 대한 discriminator loss항을 대신 이용
        # fake image를 discriminator가 1로 판정하도록 속이고자 함
        
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
        
            opt_disc = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
            # discriminator의 optimizer 정의
            # optimizer은 ADAM 적용, 설정한 learning rate 적용

            opt_gen = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
            # generator의 optimizer 정의
            # optimizer은 ADAM 적용, 설정한 learning rate 적용

            grads_disc = tf.gradients(
                disc_loss,
                tf.get_collection(
                    tf.GraphKeys.TRAINABLE_VARIABLES, scope="discriminator"))
            # discriminator의 gradient 계산
            # discriminator variable scope의 weight에 대해서만 discriminator loss적용

            grads_gen = tf.gradients(
                gen_loss, 
                tf.get_collection(
                    tf.GraphKeys.TRAINABLE_VARIABLES, scope="generator"))
            # generator의 gradient 계산
            # generator variable scope의 weight에 대해서만 generator loss적용

            grads_disc = list(zip(grads_disc, tf.get_collection(
                    tf.GraphKeys.TRAINABLE_VARIABLES, scope="discriminator")))
                    # discriminator의 gradient를 각 웨이트값과 매칭(apply gradient하기 위함)
            grads_gen = list(zip(grads_gen, tf.get_collection(
                    tf.GraphKeys.TRAINABLE_VARIABLES, scope="generator")))
                    # generator의 gradient를 각 웨이트값과 매칭(apply gradient하기 위함)

            op_disc = opt_disc.apply_gradients(grads_disc)
            # discriminator에 대하여 gradient적용하여 최적화
            op_gen = opt_gen.apply_gradients(grads_gen)
            # generator에 대하여 gradient 적용하여 최적화

        accuracy = tf.metrics.accuracy(
            labels=tf.zeros(shape=[BATCH_SIZE], dtype=tf.float32),
            predictions=tf.cast((disc_fake > 0.5),tf.float32),
            name='acc_op')
        # accuracy 계산, discriminator가 fake를 0.5 이하로 판정한 정확도

                
        saver = tf.train.Saver()
        # checkpoint 저장할 Saver객체 생성
        
        with tf.Session() as sess:
        # Session객체 생성, 컨텍스트 매니저로 자원 관리
            
            sess.run(tf.global_variables_initializer())
            # global weight 초기화
            
            sess.run(tf.local_variables_initializer())
            # local weight 초기화

            for step in range(TRAINING_STEPS):
                
                step += 1
                
                disc_loss_now, gen_loss_now, accuracy_now, _, _ = sess.run(
                    [disc_loss, gen_loss, accuracy, op_disc, op_gen])
                # discrimiantor, generator 모두 feedforward, backpropagation 실행
                                
                if (step == TRAINING_STEPS):
                    saver.save(sess, CKPT_DIR + '/gan.ckpt')
                # train 종료시 학습된 weight에 대한 checkpoint 저장
                                
                if (step % 1000 == 0):
                    print('steps: {}/{}, disc_loss: {:.4f}, gen loss: {:.4f}, accuracy: {:.4f}'.format(
                        step, TRAINING_STEPS, disc_loss_now, gen_loss_now, accuracy_now[0]))
                    # 학습과정 로깅

### Fashion-MNIST 데이터 로드 및 전처리
- Fashion-MNIST 데이터 로드
- normalize
- flatten

In [ ]:
x_train = (tf.keras.datasets.fashion_mnist.load_data()[0][0] - 127.5) / 127.5
# Fashion-MNIST 데이터 로드 (keras.datesets 모듈 이용)
# Fashion-MNIST 데이터 normalize (0~1로 mapping)

x_train = x_train.reshape([-1, 28, 28, 1]).astype(np.float32)
# Fashion-MNIST 데이터 vectorization

### GAN 학습

In [ ]:
train(x_train)

### Random Plotting 함수 정의

In [ ]:
def random_25_image_plot(seed=None):
    
    with graph.as_default():
        
        np.random.seed(seed)
        random_noise = np.random.normal(size=[25, NOISE_DIM]).astype(np.float32)
        # random noise 생성
        
        random_noise_input = train_input_fn(random_noise, batch_size=25)
        # random noise tensor 생성
        
        random_gen = gen_model(random_noise_input) * 127.5 + 127.5
        # random noise를 generator에 적용, Fashion-MNIST image 차원으로 임베딩

        fig = plt.figure(figsize=(10, 10))
        gs = gridspec.GridSpec(5, 5)
        gs.update(wspace=0.05)
        # 플로팅 사이즈, 배열, 간격 정의
        
        saver = tf.train.Saver()
        # weight restoring 진행할 Saver 객체 생성
        
        with tf.Session() as sess:
            
            saver.restore(sess, tf.train.latest_checkpoint(CKPT_DIR))
            # checkpoint로부터 weight 복원
            
            random_image = sess.run(random_gen)
            # random generation operation 진행
            
            random_image = random_image.reshape([-1, 28, 28])
            # tensorization
            
            for i in range(25):
                plt.subplot(gs[i])
                plt.axis('off')
                plt.imshow(random_image[i], cmap = 'gray')
            # plotting

### Random Plot

In [ ]:
random_25_image_plot()